# Example notebook rbatools -- E.coli

Here we exemplify rbatools with the existing RBA-model of E.coli (https://www.sciencedirect.com/science/article/abs/pii/S1096717619300710?via%3Dihub).

Please take into account, that the E.coli model takes significantly longer to execute, compared to the B.subtilis model. For speed purposes you might consider using the notebook "Example_workflows_rba_tools" instead of this one.

For further information on rba, please consider our website: https://rba.inrae.fr 

In [ ]:
# doing necessary imports:

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn
import pandas
import numpy
import rba

from ipywidgets import IntProgress
from IPython.display import display

from rbatools.rba_session import SessionRBA


# General instructions on this notebook

## Starting of session
Here we initiate an rbatools.rba_session.SessionRBA with a model of E.coli. Variable 'model_filename' specifies where the RBA model is located and should be changed by the user. 

Upon initiation of session explicit exchange reactions for external metabolites are added, to facilitate conveinient acces to metabolite exchange rates.

#### 'Bacterial-RBA-models' directory with different bacterial RBA models can be obtained from: https://github.com/SysBioInra/Bacterial-RBA-models

#### --> Relevant rbatools.rba_ression.SessionRBA methods on used here, are '__init__' and 'add_exchange_reactions'.

In [ ]:
print(SessionRBA.__doc__)

In [ ]:
print(SessionRBA.__init__.__doc__)

In [ ]:
print(SessionRBA.add_exchange_reactions.__doc__)

In [ ]:
model_filename = '../../../Bacterial-RBA-models/Escherichia-coli-K12-WT'
# initiate RBA-session with cplex as lp solver. If cplex is not available glpk can be used as lp solver.
#However glpk is significantly slower
Simulation = SessionRBA(model_filename,lp_solver="cplex")
#Simulation = SessionRBA(model_filename,lp_solver="swiglpk")
Simulation.add_exchange_reactions()

## Simulation-results handling
Here we perform simple simulation runs and record the corresponding set of parameters and results.
After we have performed all desired simulations (and did the corresponding recordings) we write them to Simulation results object. From those objects the results and parameters can be exported into various formats.
#### --> Relevant rbatools.rba_session.SessionRBA methods used here, are 'find_max_growth_rate', 'record_results', 'record_parameters' and 'write_results'.

In [ ]:
print(Simulation.find_max_growth_rate.__doc__)

In [ ]:
print(Simulation.record_results.__doc__)

In [ ]:
print(Simulation.record_parameters.__doc__)

In [ ]:
print(Simulation.write_results.__doc__)

In [ ]:
Simulation.find_max_growth_rate()
Simulation.record_results(run_name="TestRun_1")
Simulation.record_parameters(run_name="TestRun_1")

Simulation.find_max_growth_rate()
Simulation.record_results(run_name="TestRun_2")
Simulation.record_parameters(run_name="TestRun_2")

In [ ]:
Simulation.write_results(session_name="TestSession")

## Simulation-results export
Here we export results and parameters into various formats, from the previously written data.
#### --> Relevant rbatools.rba_SimulationData.RBA_SimulationData methods used here, are 'export_sbtab', 'export_csv', 'export_escher_map' and 'export_proteo_map'.

In [ ]:
print(Simulation.SimulationData.export_sbtab.__doc__)

In [ ]:
print(Simulation.SimulationData.export_csv.__doc__)

In [ ]:
print(Simulation.SimulationData.export_escher_map.__doc__)

In [ ]:
print(Simulation.SimulationData.export_proteo_map.__doc__)

In [ ]:
Simulation.SimulationData.export_sbtab(filename="../../../Test_SimulationResults_SBtab_Ecoli")
Simulation.SimulationData.export_csv(output_directory="../../../")
Simulation.SimulationData.export_escher_map(type="fluxes",output_directory="../../../")
Simulation.SimulationData.export_escher_map(type="investment",output_directory="../../../")
Simulation.SimulationData.export_proteo_map(type='genes',output_directory="../../../")
Simulation.SimulationData.export_proteo_map(type='isoforms',output_directory="../../../")

In [ ]:
print(Simulation.SimulationParameters.export_sbtab.__doc__)

In [ ]:
Simulation.SimulationParameters.export_sbtab(filename="../../../Test_SimulationParameters_SBtab_Ecoli")

## Glucose screen
Here we iterate through a predefined list of glucose concentrations (glcs) and determine the corresponding maximum growth-rates and optimal configuration of exchange fluxes. Maximum growth-rate vs. glucose results in  a Monod curve
#### --> Relevant rbatools.rba_session.SessionRBA methods used here, are 'set_medium', 'find_max_growth_rate' and 'return_exchange_fluxes'.

In [ ]:
print(Simulation.reload_model.__doc__)

In [ ]:
print(Simulation.set_medium.__doc__)

In [ ]:
print(Simulation.find_max_growth_rate.__doc__)

In [ ]:
print(Simulation.return_exchange_fluxes.__doc__)

In [ ]:
Simulation.reload_model()

glcs=[0.005+(i*0.0075) for i in range(9)]
Results_glc_screen={"Mu":[],"M_glc__D":[],"M_o2":[],"M_ac":[]}
f = IntProgress(min=0, max=len(glcs))
display(f)
for glc_conc in glcs:
    f.value += 1    
    Simulation.set_medium({'M_glc__D':glc_conc})
    mumax=Simulation.find_max_growth_rate(max=1.0,feasible_stati=["optimal","feasible","feasible_only_before_unscaling"],try_unscaling_if_sol_status_is_feasible_only_before_unscaling=False)
    J_ex=Simulation.return_exchange_fluxes()
    for i in Results_glc_screen.keys():
        if i =="Mu":
            Results_glc_screen["Mu"].append(mumax)
        else:
            if i in J_ex.keys():
                Results_glc_screen[i].append(J_ex[i])
            else:
                Results_glc_screen[i].append(0)

In [ ]:
fig = plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=5,alpha=0.6,color="dimgray")
ax.legend(["Monod curve"],loc="upper left",fontsize=12,frameon=False)
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=13,alpha=0.6,color="gainsboro")
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=10,alpha=0.6,color="lightgray")
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=7,alpha=0.6,color="silver")
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=5,alpha=0.6,color="gray")
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=2,alpha=0.6,color="dimgray")
ax.plot(glcs,Results_glc_screen["Mu"],linewidth=1,alpha=0.6,color="black")
ax2=ax.twinx()
ax2.plot(glcs,[i for i in Results_glc_screen["M_ac"]],linewidth=2,color="mediumseagreen",alpha=0.9)
ax2.plot(glcs,[-i for i in Results_glc_screen["M_o2"]],linewidth=2,color="indianred",alpha=0.9,linestyle=(0, (1, 1)))
ax2.plot(glcs,[-i for i in Results_glc_screen["M_glc__D"]],linewidth=2,color="steelblue",alpha=0.9,linestyle=(0, (3, 1, 1, 1)))

ax2.legend(["$Acetate$ excretion","$O_2$ uptake","$Glucose$ uptake"],loc="lower right",fontsize=12,frameon=False)

ax.set_title("Glucose dependent growth-rate and exchange fluxes",fontsize=15)
ax.set_xlabel("Glucose concentration (mM)",fontsize=12)
#ax.set_ylim(-0.01,0.79)
#ax.set_xlim(0.0,0.1)
ax.tick_params(axis='y', labelcolor="dimgrey")
ax2.tick_params(axis='y', labelcolor="black")
ax.set_ylabel("Growth rate ($h^{-1}$)",color="dimgrey",fontsize=12)
ax2.set_ylabel('Exchange fluxes ($\\frac{mmol}{h \\times g_{DW}}$)',color="black",fontsize=12)
#ax2.set_ylim(-0.2,15.6)
plt.tight_layout()
plt.show()



## Variability Analysis on substrate exchange fluxes
Here we iterate through a predefined list of growth rates (between 0 and the maximum wild-type growth rate) and determine the corresponding feasible ranges of glucose- and oxygen uptake. 
#### --> Relevant rbatools.rba_session.SessionRBA methods used here, are 'set_medium', 'set_growth_rate' and 'get_feasible_range'.

In [ ]:
print(Simulation.get_feasible_range.__doc__)

In [ ]:
Simulation.reload_model()

Simulation.set_medium({'M_glc__D':0.02})
mumax=Simulation.find_max_growth_rate()
Mus=[mumax*i for i in [0.001,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.95,0.975,0.99,0.995,0.9975,1]]
Reactions_to_check=["R_EX_glc__D_e","R_EX_o2_e"]
Feasible_Ranges=dict(zip(Reactions_to_check,[{"Minima":[],"Maxima":[]},{"Minima":[],"Maxima":[]},{"Minima":[],"Maxima":[]}]))
f = IntProgress(min=0, max=len(Mus))
display(f)
for mu in Mus:
    f.value += 1    
    Simulation.set_growth_rate(mu)
    Feasible_range=Simulation.get_feasible_range(variables=Reactions_to_check)
    for rxn in Reactions_to_check:
        Feasible_Ranges[rxn]["Minima"].append(Feasible_range[rxn]['Min'])
        Feasible_Ranges[rxn]["Maxima"].append(Feasible_range[rxn]['Max'])

In [ ]:
fig=plt.figure(figsize=(8,4))

ax = fig.add_subplot(111)

ax.fill_between(x=Mus,y1=Feasible_Ranges["R_EX_glc__D_e"]["Minima"],y2=Feasible_Ranges["R_EX_glc__D_e"]["Maxima"],interpolate=True,color='steelblue',alpha=0.25)
ax.plot(Mus,Feasible_Ranges["R_EX_glc__D_e"]["Minima"],color='steelblue',alpha=0.9,linewidth=2,linestyle=(0, (3, 1, 1, 1)))
ax.plot(Mus,Feasible_Ranges["R_EX_glc__D_e"]["Maxima"],color='steelblue',alpha=0.9,linewidth=2,linestyle=(0, (3, 1, 1, 1)))
ax2=ax.twinx()
ax2.fill_between(x=Mus,y1=Feasible_Ranges["R_EX_o2_e"]["Minima"],y2=Feasible_Ranges["R_EX_o2_e"]["Maxima"],interpolate=True,color='indianred',alpha=0.25)
ax2.plot(Mus,Feasible_Ranges["R_EX_o2_e"]["Minima"],color='indianred',alpha=0.9,linewidth=2,linestyle=(0, (1, 1)))
ax2.plot(Mus,Feasible_Ranges["R_EX_o2_e"]["Maxima"],color='indianred',alpha=0.9,linewidth=2,linestyle=(0, (1, 1)))

ax.tick_params(axis='y', labelcolor="royalblue")
ax2.tick_params(axis='y', labelcolor="indianred")
#ax.set_ylim(0.5,7.25)
#ax.set_yticks([1,2,3,4,5,6,7])
#ax.set_xlim(0,0.335)
#ax2.set_ylim(1,24)
#ax2.set_yticks([5,10,15,20])
#ax2.set_xlim(0,0.335)
#ax.set_xticks([0.05,0.1,0.15,0.2,0.25,0.3])
#ax2.set_xticks([0.05,0.1,0.15,0.2,0.25,0.3])
#ax.set_xticklabels([0.05,0.1,0.15,0.2,0.25,0.3])
#ax2.set_xticklabels([0.05,0.1,0.15,0.2,0.25,0.3])

ax.set_ylabel('$Glucose$ $\\frac{mmol}{h \\times g_{DW}}$)',fontsize=12,color="steelblue")
ax2.set_ylabel('$O_2$ $\\frac{mmol}{h \\times g_{DW}}$)',fontsize=12,color="indianred")

plt.title('Feasible uptake fluxes, over (sub-optimal) growth-rates',fontsize=15)
ax.set_xlabel('Growth-rate ($h^{-1}$)',fontsize=12)

plt.tight_layout()
plt.show()


## Kapp sampling - global sensitivity analysis
Here we sample N global sets of enzyme efficiencies, by drawing multiplicative factors for each enzyme's forward- and backward efficiency form a log-normal distribution (base: e). The wild-type efficiencies are then multiplied with this factor and the maximum growth-rate and corresponding exchange fluxes of glucose, oxygen and acetate are determined. 

#### --> Relevant rbatools.rba_session.SessionRBA methods used here, are 'add_parameter_multipliers_for_enzyme_efficiencies', 'set_medium' and 'sample_kapp_multipliers'.

In [ ]:
print(Simulation.add_parameter_multipliers_for_enzyme_efficiencies.__doc__)

In [ ]:
print(Simulation.sample_kapp_multipliers.__doc__)

In [ ]:
STDEV_SAMPLING=numpy.log(1.1)
MEAN_SAMPLING=0

fig, ax= plt.subplots(1, 1)
ax=plt.hist([numpy.e**j for j in list(numpy.random.normal(loc=MEAN_SAMPLING,scale=STDEV_SAMPLING,size=500000))],200)
#plt.xlim(0,6)
plt.title("Distribution of multiplicative scaling factors")
plt.xlabel("(Multiplicative) scaling factor")
plt.ylabel("Frequency")
plt.show()

In [ ]:
N_SAMPLES=30

Simulation.reload_model()
enzymes=Simulation.add_parameter_multipliers_for_enzyme_efficiencies()
Simulation.set_medium({'M_glc__D':0.02})

KappScreenResults=Simulation.sample_kapp_multipliers(n=N_SAMPLES,mean=MEAN_SAMPLING,stdev=STDEV_SAMPLING,enzymes=enzymes,Variables_to_record=["R_EX_glc__D_e","R_EX_ac_e","R_EX_o2_e"])



In [ ]:
df=pandas.DataFrame(KappScreenResults)

fig, ax= plt.subplots(4, 1,figsize=(8,8))

seaborn.histplot(data=df.loc[(df["Mu"]!=4)&(df["Mu"]!=None)], x="Mu",ax=ax[0],alpha=0.3,color="dimgray", kde=False,line_kws={"linewidth":4,"alpha":1},bins=19,fill=True)
ax[0].legend(["Growth rate ($h^{-1}$)"],frameon=False,loc="upper right",fontsize=13)
ax[0].set_yticks([])
ax[0].set_ylabel("")
ax[0].set_xlabel("")

seaborn.histplot(data=df.loc[(df["R_EX_o2_e"]!=None)], x="R_EX_o2_e",ax=ax[1],alpha=0.3,color="indianred", kde=False,line_kws={"linewidth":4,"alpha":1},bins=15,fill=True)
ax[1].legend(["$O_2$ uptake: ($\\frac{mmol}{h \\times g_{DW}}$)"],frameon=False,loc="upper right",fontsize=13)
ax[1].set_yticks([])
ax[1].set_ylabel("")
ax[1].set_xlabel("")

seaborn.histplot(data=df.loc[(df["R_EX_glc__D_e"]!=None)], x="R_EX_glc__D_e",ax=ax[2],alpha=0.3,color="steelblue", kde=False,line_kws={"linewidth":4,"alpha":1},bins=20,fill=True)
ax[2].legend(["$Glucose$ uptake: ($\\frac{mmol}{h \\times g_{DW}}$)"],frameon=False,loc="upper right",fontsize=13)
ax[2].set_yticks([])
ax[2].set_ylabel("")
ax[2].set_xlabel("")

seaborn.histplot(data=-df.loc[(df["R_EX_ac_e"]!=None)], x="R_EX_ac_e",ax=ax[3],alpha=0.3,color="mediumseagreen", kde=False,line_kws={"linewidth":4,"alpha":1},bins=30,fill=True)
ax[3].legend(["$Acetate$ excretion: ($\\frac{mmol}{h \\times g_{DW}}$)"],frameon=False,loc="upper right",fontsize=13)
ax[3].set_yticks([])
ax[3].set_ylabel("")
ax[3].set_xlabel("")

ax[0].set_title("Cellular state variability from sampled enzyme efficiencies",fontsize=15)

plt.tight_layout()
plt.show()


## Ribosome efficiency sampling
Here we screen the impact of modulation of ribosome-capacity on the maximum growth-rate at different glucose concentrations. At each growth rate multiplicative modulators on ribosome-capacity between 0.01 and 100 are applied and the maximum growth-rate is determined.
#### --> Relevant rbatools.rba_session.SessionRBA methods used here, are 'add_parameter_multiplier', 'setMedium' and 'screen_multipliers'.

In [ ]:
print(Simulation.add_parameter_multiplier.__doc__)

In [ ]:
print(Simulation.screen_multipliers.__doc__)

In [ ]:
Glucose_concentrations=[0.0125,0.025,0.05,0.1,1]

Simulation.reload_model()
Simulation.add_parameter_multiplier(model_parameter="ribosome_capacity",rebuild_model=True)
log_factors=[-2,-1,0,1,2]

Results={}
f = IntProgress(min=0, max=len(Glucose_concentrations))
display(f)
for glc_conc in Glucose_concentrations:
    f.value += 1    
    Simulation.set_medium({'M_glc__D':glc_conc})
    screen_results=Simulation.screen_multipliers(parameter="ribosome_capacity",factors=[10**i for i in log_factors])
    scaling_factors , growth_rates = [list(i) for i in zip(*screen_results.items())]
    Results[glc_conc]={"Scaling Factors":scaling_factors,"Growth_rates":[i["Mu"] for i in growth_rates]}



In [ ]:
color_dict={0.0125:"lightsteelblue",0.025:"orange",0.05:"lightsteelblue",0.1:"steelblue",1:"steelblue"}
alpha_dict={0.0125:0.3,0.025:0.8,0.05:0.7,0.1:0.4,1:1}

fig=plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)

log_scaling_coeffs=[numpy.log10(i) for i in Results[glc_conc]["Scaling Factors"]]
for glc_conc in reversed(Glucose_concentrations):
    if glc_conc in color_dict.keys():
        ax.scatter(log_scaling_coeffs,Results[glc_conc]["Growth_rates"],color=color_dict[glc_conc],alpha=1,s=50,edgecolor='none')
plt.legend(["$Glucose$ : {} $mM$".format(glc_conc) for glc_conc in reversed(Glucose_concentrations) if glc_conc in color_dict.keys()],fontsize=12,frameon=False)
ax.vlines(x=0,ymin=0,ymax=1.6,linestyles="dashed",color="black",linewidth=1,alpha=0.3)
for glc_conc in Glucose_concentrations:
    if glc_conc in color_dict.keys():
        ax.plot(log_scaling_coeffs,Results[glc_conc]["Growth_rates"],color=color_dict[glc_conc],alpha=alpha_dict[glc_conc])
for glc_conc in Glucose_concentrations:
    #ax.scatter(log_scaling_coeffs,Results[glc_conc]["Growth_rates"],color="red",alpha=1,s=55,edgecolor='face')
    if glc_conc in color_dict.keys():
        ax.scatter([numpy.log10(i) for i in Results[glc_conc]["Scaling Factors"]],Results[glc_conc]["Growth_rates"],color=color_dict[glc_conc],alpha=alpha_dict[glc_conc],s=50,edgecolor='none')

ax.set_title("Effect of ribosome capacity modulation on growth-rate",fontsize=15)
ax.set_xlabel("Scaling factor ribosome capacity",fontsize=12)
ax.set_ylabel("Growth-rate ($h^{-1}$)",fontsize=12)
ax.set_xticks([-2,-1,1,2,0])
ax.set_xticklabels([str("$10^{"+str(i)+"}$") for i in [-2,-1,1,2]]+[str(1)])
#ax.set_ylim(-0.02,1.5)

plt.tight_layout()
plt.show()







## Local sensitivity analysis of growth rate vs. ribosome capacity over glucose concentrations
Here we perform local sensitivity analysis of maximum growth-rate on ribosome- and glucose-uptake (PTSG-system) efficiency at different glucose concentrations. Local sensitivity is represented as the partial derivative of maximum growth-rate vs. model-parameter values at wild-type. 

Absolute sensitivity represents the the change in growth-rate per change in parameter value.
Scaled sensitivity represents the relative change in growth-rate per relative change in parameter value.
#### --> Relevant rbatools.rba_session.SessionRBA methods used here, are 'add_parameter_multiplier', 'setMedium' and 'local_sensitivity'.

In [ ]:
print(Simulation.local_sensitivity.__doc__)

In [ ]:
Simulation.reload_model()

Glucose_concentrations=[0.0125*(i+1) for i in range(8)]

Local_sensitivities={}
f = IntProgress(min=0, max=len(Glucose_concentrations))
display(f)
for glc_conc in Glucose_concentrations:
    f.value += 1    
    Simulation.set_medium({'M_glc__D':glc_conc})
    res=Simulation.local_sensitivity(parameters=["ribosome_capacity"],relative_parameter_difference=0.01)
    Local_sensitivities[glc_conc]=res




In [ ]:
fig=plt.figure(figsize=(8,4))
ax = fig.add_subplot(111)

ax.plot([glc_conc for glc_conc in Glucose_concentrations if glc_conc<=1],[Local_sensitivities[glc_conc].loc["ribosome_capacity","Scaled_Sensitivity"] for glc_conc in Glucose_concentrations if glc_conc<=1],
                color="indianred",
                linewidth=2,
                linestyle="dashed")
ax.scatter([glc_conc for glc_conc in Glucose_concentrations if glc_conc<=1],[Local_sensitivities[glc_conc].loc["ribosome_capacity","Scaled_Sensitivity"] for glc_conc in Glucose_concentrations if glc_conc<=1],
                color="indianred")
ax2=ax.twinx()
ax2.plot([glc_conc for glc_conc in Glucose_concentrations if glc_conc<=1],[Local_sensitivities[glc_conc].loc["ribosome_capacity","Absolute_Sensitivity"] for glc_conc in Glucose_concentrations if glc_conc<=1],
                color="steelblue",
                linewidth=2,
                linestyle="dashed")
ax2.scatter([glc_conc for glc_conc in Glucose_concentrations if glc_conc<=1],[Local_sensitivities[glc_conc].loc["ribosome_capacity","Absolute_Sensitivity"] for glc_conc in Glucose_concentrations if glc_conc<=1],
                color="steelblue")

ax.set_title("Local sensitivities ribosome_capacity at $WT$ capacity")
ax.set_xlabel("Glucose concentration ($mM$)", labelpad=0.1)
ax.set_ylabel("Scaled sensitivity",color="indianred")
ax2.set_ylabel('Absolute sensitivity',color="steelblue")

#ax.set_xlim(0,0.065)
#ax2.set_xlim(0,0.065)
plt.show()
